In [ ]:
#閾値を見るために、各フレームの変位を計算
"""
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# データを読み込む
file_path = "C:\\Users\\yusei\\OneDrive\\デスクトップ\\pos_20221006140811.csv"
data = pd.read_csv(file_path)

# 最初の2列を保持
first_two_columns = data.iloc[:, :2]

# 以降のデータを処理
processed_data = data.iloc[:, 2:].diff().fillna(0)  # 差分を計算

# 最初の2列と処理後のデータを結合
result_data = pd.concat([first_two_columns, processed_data], axis=1)

# 各行について変位を計算しcsvファイルに保存
result_data.to_csv("C:\\Users\\yusei\\OneDrive\\デスクトップ\\processed_data_with_displacement.csv", index=False)
"""

In [ ]:
"""
#一つのファイルに対して実行
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#腰のy方向(カメラ座標系の鉛直方向)の変位から立ち上がり動作を抽出することを試みました
#閾値は0.01
# データを読み込む
file_path = "C:\\Users\\miyazakipc\\Desktop\\酒井さんデータ\\若年者\\kinect\\EachMotion\\Subject1\\normal\\pos_20221006140811.csv"
data = pd.read_csv(file_path)

# 必要な列の抽出（時間, 識別ID, PELVISY）
time = data.iloc[:, 0]  # 時刻
pelvis_y = data.iloc[:, 3]

# 差分の計算
dy = pelvis_y.diff().fillna(0)

# 閾値設定（仮: 0.01）
threshold = 0.01

# 動作区間の判定（差分が閾値を超える区間）
moving = np.abs(dy) > threshold

# 動作開始と終了の特定
start_frame = moving.idxmax()  # 最初にTrueになるフレーム
end_frame = moving[::-1].idxmax()  # 最後にTrueになるフレーム

# データの切り出し
motion_data = data.iloc[start_frame:end_frame+1]

# 抽出された行番号を取得
extracted_indices = motion_data.index

# 抽出された行番号を表示
for idx in extracted_indices:
    print(f"{idx}行目が抽出されました")

# 差分が最大となる部分を特定
max_diff_index = dy.abs().idxmax()
print(f"{max_diff_index}行目と{max_diff_index + 1}行目の差が最大となりました")

# 切り出したデータを保存
motion_data.to_csv("C:\\Users\\miyazakipc\\Desktop\\酒井さんデータ\\若年者\\kinect_extract1\\Subject1\\kinect_extract_data.csv", index=False)
"""

In [ ]:
"""
import pandas as pd
import numpy as np

# データを読み込む
file_path = "C:\\Users\\miyazakipc\\Desktop\\酒井さんデータ\\若年者\\kinect\\EachMotion\\Subject1\\normal\\pos_20221006140828.csv"
data = pd.read_csv(file_path)

# 必要な列の抽出（時間, 識別ID, PELVISY）
time = data.iloc[:, 0]  # 時刻
pelvis_y = data.iloc[:, 3]

# 差分の計算
dy = pelvis_y.diff().fillna(0)

# 最初の3行分の引き算結果を破棄（ゼロで埋める）
dy.iloc[:3] = 0 

# 閾値設定（仮: 0.01）
threshold = 0.01

# 動作区間の判定（差分が閾値を超える行を抽出）
moving_indices = np.where(np.abs(dy) > threshold)[0]

# 抽出された行番号を表示
for idx in moving_indices:
    print(f"{idx}行目が抽出されました")

# 差分が最大となる部分を特定
max_diff_index = dy.abs().idxmax()
print(f"{max_diff_index}行目と{max_diff_index + 1}行目の差が最大となりました")

# 閾値を超えた行のみを抽出
motion_data = data.iloc[moving_indices]

# 抽出されたデータを保存
save_path = "C:\\Users\\miyazakipc\\Desktop\\酒井さんデータ\\若年者\\kinect_extract1\\Subject1\\kinect_extract_data2.csv"
motion_data.to_csv(save_path, index=False)
print(f"抽出されたデータが{save_path}に保存されました")
"""

In [11]:
#本命のコード
"""
import os
import pandas as pd
import numpy as np

# 入力フォルダと出力フォルダのパスを指定
input_folder = "C:\\Users\\miyazakipc\\Desktop\\酒井さんデータ\\高齢者実験\\kinect\\kourei_dataID\\kinect_20221127"
output_folder = 

# 出力フォルダが存在しない場合は作成
os.makedirs(output_folder, exist_ok=True)

# 閾値設定
threshold = 0.01

# 入力フォルダ内のすべてのCSVファイルを処理
for file_name in os.listdir(input_folder):
    if file_name.endswith(".csv"):
        input_path = os.path.join(input_folder, file_name)
        
        # データを読み込む
        data = pd.read_csv(input_path)

        # 必要な列の抽出（時間, 識別ID, PELVISY）
        time = data.iloc[:, 0]  # 時刻
        pelvis_y = data.iloc[:, 3]

        # 差分の計算
        dy = pelvis_y.diff().fillna(0)

        # 最初の3行分の引き算結果を破棄（ゼロで埋める）
        dy.iloc[:3] = 0 

        # 動作区間の判定（差分が閾値を超える行を抽出）
        moving_indices = np.where(np.abs(dy) > threshold)[0]

        # 抽出された行番号を表示
        print(f"\nファイル名: {file_name}")
        if moving_indices.size > 0:
            for idx in moving_indices:
                print(f"{idx}行目が抽出されました")
        else:
            print("閾値を超える動作区間はありませんでした。")

        # 差分が最大となる部分を特定
        max_diff_index = dy.abs().idxmax()
        if max_diff_index < len(dy) - 1:  # 安全チェック
            print(f"{max_diff_index}行目と{max_diff_index + 1}行目の差が最大となりました")
        else:
            print("差分が最大となる部分を特定できませんでした。")

        # 閾値を超えた行のみを抽出
        motion_data = data.iloc[moving_indices]

        # 抽出されたデータを保存
        output_path = os.path.join(output_folder, f"extracted_{file_name}")
        motion_data.to_csv(output_path, index=False)
        print(f"{file_name}の抽出データが{output_path}に保存されました")
"""


ファイル名: pos_20221117151430.csv
5行目が抽出されました
6行目が抽出されました
7行目が抽出されました
8行目が抽出されました
23行目が抽出されました
24行目が抽出されました
5行目と6行目の差が最大となりました
pos_20221117151430.csvの抽出データがC:\Users\miyazakipc\Desktop\酒井さんデータ\若年者\kinect_extract1\Subject6\extracted_pos_20221117151430.csvに保存されました

ファイル名: pos_20221117151439.csv
3行目が抽出されました
4行目が抽出されました
5行目が抽出されました
6行目が抽出されました
7行目が抽出されました
8行目が抽出されました
9行目が抽出されました
90行目が抽出されました
93行目が抽出されました
95行目が抽出されました
97行目が抽出されました
98行目が抽出されました
99行目が抽出されました
100行目が抽出されました
101行目が抽出されました
102行目が抽出されました
103行目が抽出されました
104行目が抽出されました
105行目が抽出されました
110行目が抽出されました
112行目が抽出されました
115行目が抽出されました
123行目が抽出されました
8行目と9行目の差が最大となりました
pos_20221117151439.csvの抽出データがC:\Users\miyazakipc\Desktop\酒井さんデータ\若年者\kinect_extract1\Subject6\extracted_pos_20221117151439.csvに保存されました

ファイル名: pos_20221117151529.csv
243行目が抽出されました
244行目が抽出されました
245行目が抽出されました
246行目が抽出されました
247行目が抽出されました
248行目が抽出されました
249行目が抽出されました
250行目が抽出されました
251行目が抽出されました
252行目が抽出されました
253行目が抽出されました
254行目が抽出されました
255行目が抽出されました
256行目が抽出されました
257行目が抽出されました
260行目が抽出されました

In [1]:
import os
import pandas as pd
import numpy as np

# 入力フォルダと出力フォルダのパスを指定
input_folder = "C:\\Users\\miyazakipc\\Desktop\\酒井さんデータ\\高齢者実験\\kinect\\kourei_dataID\\kinect_20221130"
output_folder = "C:\\Users\\miyazakipc\\Desktop\\酒井さんデータ\\高齢者実験\\kinect\\処理結果\\kinect_20221130"

# 出力フォルダが存在しない場合は作成
os.makedirs(output_folder, exist_ok=True)

# 閾値設定
threshold = 0.01

# 再帰的にフォルダ内のCSVファイルを処理
for root, _, files in os.walk(input_folder):
    for file_name in files:
        if file_name.endswith(".csv"):
            # 入力パスを取得
            input_path = os.path.join(root, file_name)
            
            # サブフォルダのパスを計算
            relative_path = os.path.relpath(root, input_folder)
            output_subfolder = os.path.join(output_folder, relative_path)
            os.makedirs(output_subfolder, exist_ok=True)  # サブフォルダを作成
            
            # データを読み込む
            data = pd.read_csv(input_path)

            # 必要な列の抽出（時間, 識別ID, PELVISY）
            time = data.iloc[:, 0]  # 時刻
            pelvis_y = data.iloc[:, 3]

            # 差分の計算
            dy = pelvis_y.diff().fillna(0)

            # 最初の3行分の引き算結果を破棄（ゼロで埋める）
            dy.iloc[:3] = 0 

            # 動作区間の判定（差分が閾値を超える行を抽出）
            moving_indices = np.where(np.abs(dy) > threshold)[0]

            # 抽出された行番号を表示
            print(f"\nファイル名: {file_name}")
            if moving_indices.size > 0:
                for idx in moving_indices:
                    print(f"{idx}行目が抽出されました")
            else:
                print("閾値を超える動作区間はありませんでした。")

            
            # 差分が最大となる部分を特定
            max_diff_index = dy.abs().idxmax()
            if max_diff_index < len(dy) - 1:  # 安全チェック
                print(f"{max_diff_index}行目と{max_diff_index + 1}行目の差が最大となりました")
            else:
                print("差分が最大となる部分を特定できませんでした。")

            # 閾値を超えた行のみを抽出
            motion_data = data.iloc[moving_indices]

            # 抽出されたデータを保存
            output_path = os.path.join(output_subfolder, f"extracted_{file_name}")
            motion_data.to_csv(output_path, index=False)
            print(f"{file_name}の抽出データが{output_path}に保存されました")



ファイル名: pos_20221130095339_ID_3.csv
100行目が抽出されました
101行目が抽出されました
102行目が抽出されました
103行目が抽出されました
104行目が抽出されました
105行目が抽出されました
106行目が抽出されました
107行目が抽出されました
108行目が抽出されました
109行目が抽出されました
146行目が抽出されました
147行目が抽出されました
148行目が抽出されました
149行目が抽出されました
150行目が抽出されました
151行目が抽出されました
152行目が抽出されました
153行目が抽出されました
154行目が抽出されました
155行目が抽出されました
156行目が抽出されました
153行目と154行目の差が最大となりました
pos_20221130095339_ID_3.csvの抽出データがC:\Users\miyazakipc\Desktop\酒井さんデータ\高齢者実験\kinect\処理結果\kinect_20221130\1-107\extracted_pos_20221130095339_ID_3.csvに保存されました

ファイル名: pos_20221130095356_ID_3.csv
53行目が抽出されました
188行目が抽出されました
189行目が抽出されました
190行目が抽出されました
191行目が抽出されました
192行目が抽出されました
193行目が抽出されました
194行目が抽出されました
195行目が抽出されました
196行目が抽出されました
197行目が抽出されました
227行目が抽出されました
228行目が抽出されました
229行目が抽出されました
230行目が抽出されました
231行目が抽出されました
232行目が抽出されました
233行目が抽出されました
234行目が抽出されました
235行目が抽出されました
236行目が抽出されました
237行目が抽出されました
239行目が抽出されました
196行目と197行目の差が最大となりました
pos_20221130095356_ID_3.csvの抽出データがC:\Users\miyazakipc\Desktop\酒井さんデータ\高齢者実験\kinect\処理結果\kinect_20221130\1-107\e